In [9]:
import json

from src.utils.helpers import load_doc_transactions_bankinter, listar_archivos_en_carpeta, save_df_by_dates
import ipywidgets as widgets
from IPython.display import display

In [10]:
# Especificar la carpeta
carpeta = 'movimientos/'  # Reemplaza esto con la ruta a tu carpeta

# Obtener los archivos y sus paths
archivos, archivos_paths = listar_archivos_en_carpeta(carpeta)

# Crear el widget Dropdown
dropdown_archivos = widgets.Dropdown(
    options=list(zip(archivos, archivos_paths)),
    description='Archivos:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

# Mostrar el widget Dropdown
display(dropdown_archivos)


Dropdown(description='Archivos:', layout=Layout(width='50%'), options=(('movimientos(1).xls', 'movimientos/mov…

In [29]:
file_categorias = "categorias.json"
df = load_doc_transactions_bankinter(dropdown_archivos.value)
print(f"cargado documento {dropdown_archivos.value} para convertir")
df.reset_index(drop=True, inplace=True)

cargado documento movimientos/movimientos(2).xls para convertir


In [30]:
comercio_categoria_dict = json.load(open(file_categorias))

In [31]:
# Función para clasificar el comercio
def clasificar_comercio(comercio):
    for key in comercio_categoria_dict:
        if key.lower() in comercio.lower():
            return comercio_categoria_dict[key]
    return 'otros'

# Crear la columna 'Tipo de Comercio' inicial
df['Tipo de Comercio'] = df['COMERCIO/CAJERO'].apply(clasificar_comercio)

# Widgets interactivos para actualizar categorías
tipo_opciones = ['bar', 'restaurante', 'desayuno', 'supermercado','ropa','servicios de internet','vacaciones', 'otros', 'tabaco', 'cosmético', 'muebles','deporte','retirada efectivo','gasolina']
widgets_dict = {}

def actualizar_categoria(change):
    indice = change['owner'].indice
    nuevo_tipo = change['new']
    comercio = df.at[indice, 'COMERCIO/CAJERO']
    comercio_categoria_dict[comercio] = nuevo_tipo
    df['Tipo de Comercio'] = df['COMERCIO/CAJERO'].apply(clasificar_comercio)
    display(df)

for i in range(len(df)):
    dropdown = widgets.Dropdown(options=tipo_opciones, value=df.at[i, 'Tipo de Comercio'])
    dropdown.indice = i
    dropdown.observe(actualizar_categoria, names='value')
    widgets_dict[i] = dropdown

In [32]:
display(widgets.VBox([widgets.HBox([widgets.Label(f"{df.at[i, 'COMERCIO/CAJERO']}:"), widgets_dict[i]]) for i in range(len(df))]))
display(df)

,FECHA,COMERCIO/CAJERO,MOVIMIENTO A,IMPORTE,Tipo de Comercio
0,2024-05-31 00:00:00,COM. USO REDES INTERNACION.,Crédito,-0.56,otros
1,2024-05-31 00:00:00,FARMACIA EL CAMINO,Crédito,-9.52,otros
2,2024-05-31 00:00:00,OPENAI *CHATGPT SUBSCR,Crédito,-18.54,servicios de internet
3,2024-06-02 00:00:00,MATILDE PATRIZIA CHIARINI,Crédito,-11.70,restaurante
4,2024-06-02 00:00:00,AGUERE CULTURAL,Crédito,-18.00,bar
5,2024-06-03 00:00:00,PIZZADILLO,Crédito,-20.00,restaurante
6,2024-06-02 00:00:00,PAYPAL *GLOVO,Crédito,-27.69,restaurante
7,2024-06-05 00:00:00,FRUTERIA MANOLO QUINTANA,Crédito,-2.73,supermercado
8,2024-06-05 00:00:00,SUMA,Crédito,-4.40,supermercado
9,2024-06-06 00:00:00,EL MOMENTO,Crédito,-3.30,desayuno


In [33]:
# guardar cambios en categorías
with open(file_categorias, 'w') as json_file:
    json.dump(comercio_categoria_dict, json_file, indent=4)
print("Diccionario guardado en comercio_categoria_dict.json")

Diccionario guardado en comercio_categoria_dict.json


In [34]:
save_df_by_dates(df)

Datos guardados en la hoja: 2024-05-31_a_2024-06-12
